## MVP Engenharia de Dados - Notebook 3: Camada GOLD

**Objetivo:** Criar o modelo de dados analítico final e demonstrar a junção de múltiplos conjuntos de dados.

**Processo de Modelagem (T e L):**

Neste notebook finalizo o pipeline de ELT, realizando as transformações analíticas finais (T) e carregando (L) o modelo de consumo.

1. **Leitura:** Lidas as três tabelas da Camada Silver (`silver_fato_vendas`, `silver_dim_produto` e a nova `silver_dim_grupos`).

2. **Junção:**
    * Executei uma sequência de "LEFT JOINS" usando a chave `COD_PRODUTO`:
        1. **Vendas + Produto:** Para trazer o nome do item.
        2. **Vendas + Grupos:** Para trazer os grupos (ex: "1.01 AGUAS", "2.01 CERVEJAS"), permitindo a distinção entre comidas, bebidas...

3. **Transformações de Negócio:**
    * **Categorização Taxas:** Implementei uma regra para identificar as taxas de serviço ('VALOR DO SERVIÇO') e atribuí-las a um novo grupo chamado `TAXA DE SERVIÇO`, separando-as dos produtos de consumo real.
    * **Cálculo de Métrica:** Criei a coluna `FATURAMENTO_LIQUIDO` (calculada como `VALOR_TOTAL_ITEM` - `VALOR_DESCONTO`), pois essa é a métrica correta para a análise financeira.
    * **Classificação de Cliente:** Criei a coluna `TIPO_DE_CONSUMO` (baseada no `ID_SOCIO` nulo/vazio) para classificar as vendas entre `AVULSO_PAGO_NA_HORA` e `SOCIO_A_FATURAR`.
    * **Tratamento de Nulos:** Apliquei regras para garantir que produtos sem grupo definido apareçam como "OUTROS".
    * **Tratamento da Taxa de Serviço:** Para garantir a precisão das análises, apliquei uma regra específica para os registros de `Taxa de Serviço`:
        * Defini o `FATURAMENTO_LIQUIDO` como 0.00 para todos os itens classificados como "Taxa de Serviço". Como a taxa não é um produto (alimento/bebida), sua inclusão no faturamento líquido distorceria o ranking de rentabilidade das categorias reais do cardápio. O valor original da taxa permanece preservado na coluna `VALOR_TOTAL_ITEM`, permitindo conferências financeiras sem comprometer os indicadores de desempenho de vendas.

4. **Carga:**
    * O resultado final é salvo na tabela `default.gold_vendas_flat_model`.

In [0]:
from pyspark.sql.functions import col, when, lit

print("1. Carregando tabelas da camada Silver...")
df_fato = spark.read.table("default.silver_fato_vendas")
df_prod = spark.read.table("default.silver_dim_produto")
df_grup = spark.read.table("default.silver_dim_grupos")

print("2. Realizando os Joins (Vendas + Produto + Grupos)...")

# LEFT JOIN para não perder vendas mesmo se o produto não tiver cadastro
df_gold = df_fato.join(df_prod, "COD_PRODUTO", "left") \
                 .join(df_grup, "COD_PRODUTO", "left")


print("3. Aplicando Regras de Negócio...")

# Categorização (Tratamento de Nulos e Taxas)
df_gold = df_gold.withColumn("NOME_GRUPO", 
    when(col("NOME_PRODUTO") == "VALOR DO SERVIÇO", "TAXA DE SERVIÇO")
    .when(col("NOME_GRUPO").isNull(), "OUTROS")
    .otherwise(col("NOME_GRUPO"))
)

# Tratamento para produtos que existem na venda (Fato), mas não no cadastro
df_gold = df_gold.withColumn("NOME_PRODUTO", 
    when(col("NOME_PRODUTO").isNull(), "PRODUTO NAO CADASTRADO")
    .otherwise(col("NOME_PRODUTO"))
)

# Faturamento Líquido (O valor real que entrou no caixa)
# Aqui usamos o VALOR_TOTAL_ITEM e o VALOR_DESCONTO que vêm da df_fato
df_gold = df_gold.withColumn("FATURAMENTO_LIQUIDO", col("VALOR_TOTAL_ITEM") - col("VALOR_DESCONTO"))

# Classificação de Cliente (Sócio vs Avulso)
df_gold = df_gold.withColumn("TIPO_PAGAMENTO", 
                             when((col("ID_SOCIO").isNull()) | (col("ID_SOCIO") == ""), "AVULSO_PAGO_NA_HORA")
                             .otherwise("SOCIO_A_FATURAR"))


# Selecionar as colunas
colunas_finais = [
    "NUM_NFCE", "DATA_HORA", "NOME_PDV", 
    "COD_PRODUTO", "NOME_PRODUTO", "NOME_GRUPO", 
    "QUANTIDADE", "PRECO_UNITARIO", 
    "VALOR_TOTAL_ITEM",
    "VALOR_DESCONTO",
    "FATURAMENTO_LIQUIDO",
    "ID_SOCIO", "TIPO_PAGAMENTO"
]

df_gold_final = df_gold.select(*colunas_finais)

print("4. Salvando tabela única (Flat Table) na Gold...")

df_gold_final.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("default.gold_vendas_flat_model")

print("\n>>> SUCESSO! Tabela 'default.gold_vendas_flat_model' gerada com todas as métricas financeiras!")
display(df_gold_final.limit(10))